# Preparando, definindo bibliotecas utilizadas para manipulação dos dados

In [11]:
#importando bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#variaveis de working (auxiliares)
wrk_var_milhao = 1.e6

#ajustando formatação de valor decimal para visualizar melhor
pd.options.display.float_format = '{:,.2f}'.format

# Carga de dados sobre quantidade populacional por unidade federativa, leitura direto por link em site wikipedia (dados em tabela, no arquivo html)

In [20]:
#carga dos dados de populacao - link wikipedia
uri_populacao = 'https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o'

html_content = requests.get(uri_populacao).text
soup = BeautifulSoup(html_content, "lxml")
tabela = soup.find_all('table')[0]
dados_tabela = pd.read_html(str(tabela))
dados_populacao = pd.DataFrame(dados_tabela[0])
dados_populacao

,Posição,Unidade federativa,População,% da pop. total,País comparável (habitantes)
0,1,São Paulo,46 289 333,"21,9%",Espanha (46 439 864)
1,2,Minas Gerais,21 292 666,"10,1%",Sri Lanka (20 675 000)
2,3,Rio de Janeiro,17 366 189,"8,2%",Países Baixos (16 922 900)
3,4,Bahia,14 930 634,"7,1%",Chade (14 037 000)
4,5,Paraná,11 516 840,"5,4%",Bolívia (11 410 651)
5,6,Rio Grande do Sul,11 422 973,"5,4%",Bélgica (11 250 659)
6,7,Pernambuco,9 616 621,"4,5%",Bielorrússia (9 485 300)
7,8,Ceará,9 187 103,"4,3%",Emirados Árabes Unidos (9 157 000)
8,9,Pará,8 690 745,"4,1%",Áustria (8 602 112)
9,10,Santa Catarina,7 252 502,"3,4%",Sérvia (7 114 393)


# Preparação de base com informações de quantidade populacional por unidade federativa

In [29]:
#formatacao dataframe apenas com as colunas que interessam para o estudo
populacao = dados_populacao[["Unidade federativa" , "População"]]
populacao.rename(columns={'Unidade federativa':'uf', 'População':'populacao'}, inplace=True)
populacao = populacao.set_index("uf")
populacao["populacao"] = populacao["populacao"].str.replace(" " , "")
populacao["populacao"] = populacao["populacao"].astype("int64")
display(populacao)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,populacao
uf,
São Paulo,46289333
Minas Gerais,21292666
Rio de Janeiro,17366189
Bahia,14930634
Paraná,11516840
Rio Grande do Sul,11422973
Pernambuco,9616621
Ceará,9187103
Pará,8690745


# Carga de dados com os valores aprovados para internações hospitalares, separados por unidades federativas

In [2]:
#carga dos dados de valores totais aprovados
uri_valor_aprovado="https://raw.githubusercontent.com/victorscouto/bootcamp-alura-datascience/main/modulo_02/dados/valores_aprovados_gastos.csv"
dados_valor_aprovado=pd.read_csv(uri_valor_aprovado,
                              encoding= "ISO-8859-1",
                              skiprows= 3,
                              sep=";",
                              skipfooter= 12,
                              thousands=".",
                              decimal=",",
                              engine="python")

# Preparação da base com os gastos mais recentes

In [10]:
gastos_mais_recentes = dados_valor_aprovado[["Unidade da Federação" , "2021/Mar"]]
gastos_mais_recentes["Unidade da Federação"] = gastos_mais_recentes["Unidade da Federação"].str.slice(start=3)
gastos_mais_recentes = gastos_mais_recentes.set_index("Unidade da Federação")
gastos_mais_recentes.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,2021/Mar
Unidade da Federação,
Rondônia,"4,732,441.04"
Acre,"1,137,341.65"
Amazonas,"6,799,060.80"
Roraima,"2,381,844.31"
Pará,"13,069,564.31"


#Cruzando as bases Populacao x Gastos recentes

In [30]:
#cruzando as bases de gastos recentes e quantidade populacao por UF
populacao.join(gastos_mais_recentes)

,populacao,2021/Mar
uf,,
São Paulo,46289333,"177,408,317.71"
Minas Gerais,21292666,"79,780,838.00"
Rio de Janeiro,17366189,"49,149,884.48"
Bahia,14930634,"31,314,602.44"
Paraná,11516840,"43,480,729.07"
Rio Grande do Sul,11422973,"37,798,832.80"
Pernambuco,9616621,"39,472,738.81"
Ceará,9187103,"17,173,151.55"
Pará,8690745,"13,069,564.31"
